<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/NB_Scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multinomial Naive Bayes

## Importing Data

In [1]:
# mount Google Drive
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [2]:
# import file from Google Drive
import pandas as pd
df = pd.read_csv('/drive/My Drive/book32listing.csv',encoding='latin1', header=None)

In [3]:
# drop columns that are not needed
#df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6,5]] # only columns with titles and genres
df1.columns = ['title', 'genre', 'label']
print(df1)

                                                    title      genre  label
0                         Mom's Family Wall Calendar 2016  Calendars      3
1                         Doug the Pug 2016 Wall Calendar  Calendars      3
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars      3
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars      3
4                    Sierra Club Engagement Calendar 2016  Calendars      3
...                                                   ...        ...    ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel     29
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel     29
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel     29
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel     29
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel     29

[207572 rows x 3 columns]


## Preprocessing

In [7]:
# case collapsing
df1['title'] = df1.title.map(lambda x: x.lower())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# remove punctuation
df1['title'] = df1.title.str.replace('[^\w\s]', '')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
# tokenization
df1['title'] = df1['title'].apply(nltk.word_tokenize)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# transform data into occurrences
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df1['title'] = df1['title'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df1['title'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)

## Training NB Model

In [13]:
# split data into train (80%) and test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df1['label'], test_size=0.2, random_state=69)

In [14]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

## Evaluating NB Model

In [15]:
# accuracy
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.544285198121161


In [16]:
# confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[ 343    0   60 ...  277    0    0]
 [   7   21   30 ...  265    0    0]
 [   5    0 1490 ...  101    0    0]
 ...
 [   2    0   17 ... 3426    0    0]
 [   2    0   13 ...   56    1    0]
 [   0    0   88 ...   15    0    6]]


In [ ]:
# examine class distribution
y_test.value_counts()